In [1]:
import numpy as np
import pandas as pd
import scipy
import networkx as nx
import itertools
import collections
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pickle

In [2]:
# Función para convertir variables categóricas a one-hot encoding
def convert_categorical(df_X, _X):
    values = np.array(df_X[_X])
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    df_X = df_X.drop(_X, 1)
    for j in range(integer_encoded.max() + 1):
        df_X.insert(loc=j + 1, column=str(_X) + str(j + 1), value=onehot_encoded[:, j])
    return df_X

In [3]:
alpha_coefs = [0.002] #, 0.005, 0.01, 0.015]#, 0.02, 0.025, 0.03, 0.035, 0.04]

In [4]:
# Carga de datos
df = pd.read_csv('Datos/XWines_Slim_150K_ratings.csv')
df_user = pd.read_csv('Datos/XWines_Slim_10K_users.csv')

# Conversión de variables categóricas
df_user = convert_categorical(df_user, 'Country')
df_user = convert_categorical(df_user, 'Gender')
df_user['bin'] = pd.cut(df_user['Age'], [0, 10, 20, 30, 40, 50, 100], labels=['1', '2', '3', '4', '5', '6'])
df_user['Age'] = df_user['bin']
df_user = df_user.drop('bin', axis=1)
df_user = convert_categorical(df_user, 'Age')

/tmp/ipykernel_338082/875457881.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Datos/XWines_Slim_150K_ratings.csv')
/tmp/ipykernel_338082/644169219.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_X = df_X.drop(_X, 1)
/tmp/ipykernel_338082/644169219.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_X = df_X.drop(_X, 1)
/tmp/ipykernel_338082/644169219.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_X = df_X.drop(_X, 1)


In [5]:
for alpha_coef in alpha_coefs:
    pairs = []
    grouped = df.groupby(['WineID', 'Rating'])
    for key, group in grouped:
        pairs.extend(list(itertools.combinations(group['UserID'], 2)))
    
    counter = collections.Counter(pairs)
    alpha = alpha_coef * len(df['WineID'].unique())  # Ajuste del parámetro alpha
    edge_list = map(list, collections.Counter(el for el in counter.elements() if counter[el] >= alpha).keys())

    
    # Construcción del grafo
    G = nx.Graph()

    for el in edge_list:
        G.add_edge(el[0], el[1], weight=1)
        G.add_edge(el[0], el[0], weight=1)
        G.add_edge(el[1], el[1], weight=1)

    
    # Asegurarse de que el grafo no está vacío
    if len(G.nodes()) == 0:
        print(f"Grafo vacío para alpha_coef={alpha_coef}")
        continue
    # Grafo vacio para alpha_coef=0.015

    # Guardar el grafo en un archivo
    with open(f"data_wines/graph_alpha_{alpha_coef}.pkl", "wb") as f:
        pickle.dump(G, f)
    
    # Cálculo de centralidades y normalización
    pr = nx.pagerank(G.to_directed())
    if pr:
        df_user['PR'] = df_user['UserID'].map(pr).fillna(0)
        df_user['PR'] /= float(df_user['PR'].max())
    
    dc = nx.degree_centrality(G)
    if dc:
        df_user['CD'] = df_user['UserID'].map(dc).fillna(0)
        df_user['CD'] /= float(df_user['CD'].max())
    
    cc = nx.closeness_centrality(G)
    if cc:
        df_user['CC'] = df_user['UserID'].map(cc).fillna(0)
        df_user['CC'] /= float(df_user['CC'].max())
    
    bc = nx.betweenness_centrality(G)
    if bc:
        df_user['CB'] = df_user['UserID'].map(bc).fillna(0)
        df_user['CB'] /= float(df_user['CB'].max())
    
    lc = nx.load_centrality(G)
    if lc:
        df_user['LC'] = df_user['UserID'].map(lc).fillna(0)
        df_user['LC'] /= float(df_user['LC'].max())
    
    nd = nx.average_neighbor_degree(G, weight='weight')
    if nd:
        df_user['AND'] = df_user['UserID'].map(nd).fillna(0)
        df_user['AND'] /= float(df_user['AND'].max())
    
    # Preparación de los datos finales y guardado
    X_train = df_user[df_user.columns[1:]]
    X_train.fillna(0, inplace=True)

    X_train.to_pickle(f"data_wines/x_train_alpha({alpha_coef}).pkl")

/tmp/ipykernel_338082/3115068922.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(0, inplace=True)
